In [57]:
import kuzu
import pandas as pd
import numpy as np


In [58]:
# Initialize database
db = kuzu.Database("./knowledge_graph")
conn = kuzu.Connection(db)

In [59]:
#Entity Table Creation
conn.execute("""
CREATE NODE TABLE Entity(
    entity_id STRING,
    description STRING,
    entity_properties STRING[],
    PRIMARY KEY(entity_id)
)""")

# Metrics Table Creation
conn.execute("""
CREATE NODE TABLE Metrics(
    metric_id STRING,
    metric_name STRING,
    cur_value DOUBLE,
    change DOUBLE,
    trend DOUBLE[],
    goal STRING,
    PRIMARY KEY(metric_id)
)""")

# Dimensions Table Creation
conn.execute("""
CREATE NODE TABLE Dimensions(
    dimension_id STRING,
    dimension_name STRING,
    dimension_values STRING[],
    dimension_description STRING,
    PRIMARY KEY(dimension_id)
)""")

#User Table Creation
conn.execute("""
CREATE NODE TABLE User(
    user_id STRING,
    user_name STRING,
    user_domain STRING,
    user_theme STRING,
    PRIMARY KEY(user_id)
)""")

In [60]:
#Relationships
conn.execute("CREATE REL TABLE Relates_to(FROM Entity To Entity)")
conn.execute("CREATE REL TABLE Belongs_to(FROM Dimensions to Entity)")
conn.execute("CREATE REL TABLE Has_Metrics(FROM Entity TO Metrics)")
conn.execute("CREATE REL TABLE Impacts(FROM Metrics TO Metrics, relationship STRING, strength STRING)")
conn.execute("CREATE REL TABLE Drives(FROM Dimensions TO Metrics, relationship STRING, strength STRING)")
conn.execute("CREATE REL TABLE Similar_to(FROM User TO User, similarity_strength STRING)")
conn.execute("CREATE REL TABLE Associated(FROM User TO Metrics, Relationship STRING)")


In [61]:
#Adding entities
conn.execute("COPY Entity FROM 'entity.csv'")

In [62]:
# Creating the relationships between entities
conn.execute("""
MATCH (e1:Entity {entity_id: 'CUSTOMER'}), (e2:Entity {entity_id: 'SALES'})
CREATE (e1)-[:Relates_to]->(e2)
""")
print("\nRelationships have been created between Customer and Sales.")

conn.execute("""
MATCH (e1:Entity {entity_id: 'SALES'}), (e2:Entity {entity_id: 'PRODUCT'})
CREATE (e1)-[:Relates_to]->(e2)
""")
print("\nRelationships have been created between Sales and Product.")

conn.execute("""
MATCH (e1:Entity {entity_id: 'SUPPLIER'}), (e2:Entity {entity_id: 'PRODUCT'})
CREATE (e1)-[:Relates_to]->(e2)
""")
print("\nRelationships have been created between Supplier and Product.")

conn.execute("""
MATCH (e1:Entity {entity_id: 'PRODUCT'}), (e2:Entity {entity_id: 'WAREHOUSE'})
CREATE (e1)-[:Relates_to]->(e2)
""")
print("\nRelationships have been created between Product and Warehouse.")

conn.execute("""
MATCH (e1:Entity {entity_id: 'PRODUCT'}), (e2:Entity {entity_id: 'RETURN'})
CREATE (e1)-[:Relates_to]->(e2)
""")

print("\nRelationships have been created between Product and Return.")



Relationships have been created between Customer and Sales.

Relationships have been created between Sales and Product.

Relationships have been created between Supplier and Product.

Relationships have been created between Product and Warehouse.

Relationships have been created between Product and Return.


In [63]:
#Adding metrics
conn.execute("COPY Metrics FROM 'metrics.csv'")

In [64]:
# Creating the relationships between Customer and Metrics
conn.execute("""
MATCH (c:Entity {entity_id: 'CUSTOMER'}), (m:Metrics {metric_id: 'CLTV'})
CREATE (c)-[:Has_Metrics]->(m)
""")

conn.execute("""
MATCH (c:Entity {entity_id: 'CUSTOMER'}), (m:Metrics {metric_id: 'ActiveCustomers'})
CREATE (c)-[:Has_Metrics]->(m)
""")

conn.execute("""
MATCH (c:Entity {entity_id: 'CUSTOMER'}), (m:Metrics {metric_id: 'ChurnRiskScore'})
CREATE (c)-[:Has_Metrics]->(m)
""")

print("\nRelationships have been created between Customer and Metrics.")



Relationships have been created between Customer and Metrics.


In [65]:
# Creating relationships between Product and Metrics

conn.execute("""
MATCH (p:Entity {entity_id: 'PRODUCT'}), (m:Metrics {metric_id: 'BackOrderRatio'})
CREATE (p)-[:Has_Metrics]->(m)
""")

print("\nRelationships have been created between Product and Metrics.")



Relationships have been created between Product and Metrics.


In [66]:
# Creating relationships between SALES and Metrics
conn.execute("""
MATCH (o:Entity {entity_id: 'SALES'}), (m:Metrics {metric_id: 'AOV'})
CREATE (o)-[:Has_Metrics]->(m)
""")

conn.execute("""
MATCH (o:Entity {entity_id: 'SALES'}), (m:Metrics {metric_id: 'Revenue'})
CREATE (o)-[:Has_Metrics]->(m)
""")

conn.execute("""
MATCH (o:Entity {entity_id: 'SALES'}), (m:Metrics {metric_id: 'PurchaseFrequency'})
CREATE (o)-[:Has_Metrics]->(m)
""")

conn.execute("""
MATCH (o:Entity {entity_id: 'SALES'}), (m:Metrics {metric_id: 'Recency'})
CREATE (o)-[:Has_Metrics]->(m)
""")

print("\nRelationships have been created between SALES and Metrics.")



Relationships have been created between SALES and Metrics.


In [67]:
# Creating relationships between Warehouse and Metrics
conn.execute("""
MATCH (o:Entity {entity_id: 'WAREHOUSE'}), (m:Metrics {metric_id: 'StockoutRate'})
CREATE (o)-[:Has_Metrics]->(m)
""")

conn.execute("""
MATCH (o:Entity {entity_id: 'WAREHOUSE'}), (m:Metrics {metric_id: 'AvgDeliveryVolume'})
CREATE (o)-[:Has_Metrics]->(m)
""")
conn.execute("""
MATCH (o:Entity {entity_id: 'WAREHOUSE'}), (m:Metrics {metric_id: 'AvgDeliveryCost'})
CREATE (o)-[:Has_Metrics]->(m)
""")

print("\nRelationships have been created between WAREHOUSE and Metrics.")



Relationships have been created between WAREHOUSE and Metrics.


In [68]:
# Creating relationships between Return and Metrics
conn.execute("""
MATCH (o:Entity {entity_id: 'RETURN'}), (m:Metrics {metric_id: 'ReturnRate'})
CREATE (o)-[:Has_Metrics]->(m)
""")
conn.execute("""
MATCH (o:Entity {entity_id: 'RETURN'}), (m:Metrics {metric_id: 'AvgReturnAmt'})
CREATE (o)-[:Has_Metrics]->(m)
""")

print("\nRelationships have been created between WAREHOUSE and Metrics.")



Relationships have been created between WAREHOUSE and Metrics.


In [69]:
#Adding dimensions
conn.execute("COPY Dimensions FROM 'dimensions.csv'")

In [70]:
# Creating the relationships between Dimensions and Customer 
conn.execute("""
MATCH (d:Dimensions {dimension_id: 'Segment'}),(c:Entity {Entity_id: 'CUSTOMER'})
CREATE (d)-[:Belongs_to]->(c)
""")
conn.execute("""
MATCH(d:Dimensions {dimension_id: 'Gender'}),(c:Entity {Entity_id: 'CUSTOMER'})
CREATE (d)-[:Belongs_to]->(c)
""")
conn.execute("""
MATCH (d:Dimensions {dimension_id: 'AgeGroup'}),(c:Entity {Entity_id: 'CUSTOMER'})
CREATE (d)-[:Belongs_to]->(c)
""")
conn.execute("""
MATCH (d:Dimensions {dimension_id: 'IncomeBracket'}),(c:Entity {Entity_id: 'CUSTOMER'})
CREATE (d)-[:Belongs_to]->(c)
""")

print("\nRelationships have been created between Dimensions and Customer.")



Relationships have been created between Dimensions and Customer.


In [71]:
# Creating the relationships between Product and Dimensions
conn.execute("""
MATCH (d:Dimensions {dimension_id: 'ProductSubcategory'}), (p:Entity{entity_id: 'PRODUCT'})
CREATE (d)-[Belongs_to]->(p)
""")

conn.execute("""
MATCH (d:Dimensions {dimension_id: 'ProductCategory'}), (p:Entity{entity_id: 'PRODUCT'})
CREATE (d)-[Belongs_to]->(p)
""")

conn.execute("""
MATCH  (d:Dimensions {dimension_id: 'Color'}), (p:Entity{entity_id: 'PRODUCT'})
CREATE (d)-[Belongs_to]->(p)
""")

conn.execute("""
MATCH  (d:Dimensions {dimension_id: 'Size'}), (p:Entity{entity_id: 'PRODUCT'})          
CREATE (d)-[Belongs_to]->(p)
""")

print("\nRelationships have been created between Product and Dimensions.")



Relationships have been created between Product and Dimensions.


In [72]:
# Creating the relationships between SALES and SALES_Dimensions
conn.execute("""
MATCH (d:Dimensions {dimension_id: 'Region'}), (s:Entity{entity_id: 'SALES'})
CREATE (d)-[Belongs_to]->(s)
""")

conn.execute("""
MATCH (d:Dimensions {dimension_id: 'SalesDate'}), (s:Entity{entity_id: 'SALES'})
CREATE (d)-[Belongs_to]->(s)
""")

conn.execute("""
MATCH (d:Dimensions {dimension_id: 'PaymentType'}), (s:Entity{entity_id: 'SALES'})
CREATE (d)-[Belongs_to]->(s)
""")
print("\nRelationships have been created between SALES and Dimensions.")


Relationships have been created between SALES and Dimensions.


In [73]:
# Creating the relationships between Supplier and SALES_Dimensions
conn.execute("""
MATCH (d:Dimensions {dimension_id: 'SupplierCountry'}), (s:Entity{entity_id: 'SUPPLIER'})
CREATE (d)-[Belongs_to]->(s)
""")

conn.execute("""
MATCH (d:Dimensions {dimension_id: 'SupplierName'}), (s:Entity{entity_id: 'SUPPLIER'})
CREATE (d)-[Belongs_to]->(s)
""")
print("\nRelationships have been created between SUPPLIER and Dimensions.")


Relationships have been created between SUPPLIER and Dimensions.


In [74]:
# Creating the relationships between Warehouse and SALES_Dimensions
conn.execute("""
MATCH (d:Dimensions {dimension_id: 'WarehouseLocation'}), (w:Entity{entity_id: 'WAREHOUSE'})
CREATE (d)-[Belongs_to]->(w)
""")

conn.execute("""
MATCH (d:Dimensions {dimension_id: 'StorageSection'}), (w:Entity{entity_id: 'WAREHOUSE'})
CREATE (d)-[Belongs_to]->(w)
""")
print("\nRelationships have been created between Warehouse and Dimensions.")


Relationships have been created between Warehouse and Dimensions.


In [75]:
# Creating the relationships between Warehouse and Dimensions
conn.execute("""
MATCH (d:Dimensions {dimension_id: 'ReturnDate'}), (r:Entity{entity_id: 'RETURN'})
CREATE (d)-[Belongs_to]->(r)
""")


In [76]:
conn.execute('COPY Impacts FROM "impacts.csv"')
conn.execute('COPY Drives FROM "drives.csv"')

In [77]:
conn.execute("COPY User FROM 'user.csv'")
conn.execute("COPY Similar_to FROM 'similar_to.csv'")
conn.execute('COPY Associated FROM "associated.csv"')

In [78]:
result = conn.execute("MATCH (e:Entity) RETURN e.entity_id, e.description, e.entity_properties LIMIT 10;")
result.get_as_df()


,e.entity_id,e.description,e.entity_properties
0,CUSTOMER,Customer Entity,['Some Properties']
1,PRODUCT,Product Entity,['Some Properties']
2,SALES,Sales Entity,['Some Properties']
3,RETURN,Returns Entity,['Some Properties']
4,SUPPLIER,Supplier Entity,['Some Properties']
5,WAREHOUSE,Warehouse Entity,['Some Properties']


In [79]:
result = conn.execute("MATCH (m:Metrics) RETURN m.metric_id, m.metric_name, m.cur_value, m.change, m.trend, m.goal LIMIT 10;")
result.get_as_df()


,m.metric_id,m.metric_name,m.cur_value,m.change,m.trend,m.goal
0,CLTV,CLTV,1500.00,5.00,"[1200.0, 1250.0, 1300.0, 1350.0, 1400.0, 1450....",Increase CLTV by 10%
1,PurchaseFrequency,PurchaseFrequency,4.50,2.20,"[3.0, 3.5, 4.0, 4.2, 4.3, 4.4, 4.5, 4.6, 4.6, ...",Increase purchase frequency by 1
2,ChurnRiskScore,ChurnRiskScore,0.12,-1.50,"[0.15, 0.14, 0.13, 0.12, 0.11, 0.11, 0.12, 0.1...",Reduce churn risk below 10%
3,ActiveCustomers,ActiveCustomers,1200.00,100.00,"[1100.0, 1120.0, 1130.0, 1150.0, 1180.0, 1200....",Increase active customers by 10%
4,BackOrderRatio,BackOrder Ratio,0.08,-1.50,"[0.12, 0.1, 0.09, 0.08, 0.07, 0.08, 0.09, 0.1,...",Reduce back order rate below 5%
5,AOV,AOV,135.80,12.80,"[100.0, 105.0, 110.0, 115.0, 120.0, 125.0, 130...",Increase AOV by 10%
6,Revenue,Revenue,500000.00,10000.00,"[450000.0, 460000.0, 470000.0, 480000.0, 49000...",Increase revenue by 2%
7,StockoutRate,Stockout Rate,0.05,-0.02,"[0.08, 0.07, 0.06, 0.06, 0.05, 0.05, 0.05, 0.0...",Reduce stockout rate below 5%
8,AvgReturnAmt,AverageReturnAmount,50.00,-5.00,"[55.0, 54.0, 53.0, 52.0, 51.0, 50.0, 49.0, 48....",Reduce average return amount by 10%
9,ReturnRate,ReturnRate,0.10,-3.30,"[0.15, 0.14, 0.13, 0.13, 0.12, 0.12, 0.11, 0.1...",Reduce return rate below 10%


In [80]:
result = conn.execute("MATCH (d:Dimensions) RETURN d.dimension_id, d.dimension_name, d.dimension_values, d.dimension_description LIMIT 10;")
result.get_as_df()

,d.dimension_id,d.dimension_name,d.dimension_values,d.dimension_description
0,Region,Region,"['Northwest', 'Northeast', 'Midwest', 'South',...",Regions where customers are located
1,SalesDate,SalesDate,"['2023-11-05', '2023-07-21', '2022-12-15', '20...",Date of Order
2,PaymentType,PaymentType,"['Credit Card', 'Cash', 'Check', 'Online']",Mode of Payment
3,Color,Color,"['Silver', 'Black', 'Red', 'Blue', 'White']",Available colors of the product
4,ProductCategory,ProductCategory,"['Bikes', 'Components', 'Clothing', 'Accessori...",Broad categories of products
5,ProductSubcategory,ProductSubcategory,"['Mountain Bikes', 'Road Bikes', 'Hybrid Bikes...",Subcategories under product category
6,Size,Size,"['Small', 'Medium', 'Large', 'Extra Large', 'X...",Available sizes for products
7,SupplierCountry,SupplierCountry,"['USA', 'Canada', 'Germany', 'Japan', 'China']",Countries of product suppliers
8,SupplierName,SupplierName,"['ACME Supplies', 'Global Parts Inc.', 'Qualit...",Names of product suppliers
9,Segment,Customer Segment,"['Individual', 'Corporate']",Type of customer (Individual or Corporate)


In [81]:
result = conn.execute("MATCH (u:User) RETURN u.user_id, u.user_name, u.user_domain, u.user_theme LIMIT 10;")
result.get_as_df()


,u.user_id,u.user_name,u.user_domain,u.user_theme
0,1,a,Sales,Maximise User Value
1,2,b,Sales,Maximise User Value
2,3,c,Sales,Maximise User Value
3,4,d,Sales,Maximise User Value
4,5,e,Marketing,Maximise customer lifetime value
5,6,f,Marketing,Increase Active customers


In [82]:
result = conn.execute("MATCH (e1:Entity)-[r:Relates_to]->(e2:Entity) RETURN e1.entity_id, e2.entity_id LIMIT 10;")
result.get_as_df()


,e1.entity_id,e2.entity_id
0,CUSTOMER,SALES
1,PRODUCT,WAREHOUSE
2,PRODUCT,RETURN
3,SALES,PRODUCT
4,SUPPLIER,PRODUCT


In [83]:
result = conn.execute("MATCH (e:Entity)-[r:Has_Metrics]->(m:Metrics) RETURN e.entity_id, m.metric_id;")
result.get_as_df()


,e.entity_id,m.metric_id
0,CUSTOMER,CLTV
1,SALES,PurchaseFrequency
2,CUSTOMER,ChurnRiskScore
3,CUSTOMER,ActiveCustomers
4,PRODUCT,BackOrderRatio
5,SALES,AOV
6,SALES,Revenue
7,WAREHOUSE,StockoutRate
8,RETURN,AvgReturnAmt
9,RETURN,ReturnRate


In [84]:
result = conn.execute("MATCH (d:Dimensions)-[r:Belongs_to]->(e:Entity) RETURN d.dimension_id, e.entity_id;")
result.get_as_df()

,d.dimension_id,e.entity_id
0,Region,SALES
1,SalesDate,SALES
2,PaymentType,SALES
3,Color,PRODUCT
4,ProductCategory,PRODUCT
5,ProductSubcategory,PRODUCT
6,Size,PRODUCT
7,SupplierCountry,SUPPLIER
8,SupplierName,SUPPLIER
9,Segment,CUSTOMER


In [85]:
result = conn.execute("MATCH (m1:Metrics)-[r:Impacts]->(m2:Metrics) RETURN m1.metric_id,r.relationship, r.strength, m2.metric_id ;")
result.get_as_df()


,m1.metric_id,r.relationship,r.strength,m2.metric_id
0,CLTV,Direct,Strong,PurchaseFrequency
1,CLTV,Direct,Moderate,AvgDeliveryVolume
2,CLTV,Inverse,Strong,ChurnRiskScore
3,CLTV,Direct,Moderate,Recency
4,PurchaseFrequency,Direct,Strong,Revenue
5,PurchaseFrequency,Inverse,Strong,ChurnRiskScore
6,PurchaseFrequency,Direct,Moderate,ReturnRate
7,PurchaseFrequency,Direct,Moderate,Recency
8,ChurnRiskScore,Inverse,Moderate,ActiveCustomers
9,ChurnRiskScore,Inverse,Moderate,Recency


In [86]:
result = conn.execute("MATCH (d:Dimensions)-[r:Drives]->(m:Metrics) RETURN d.dimension_id, m.metric_id, r.relationship, r.strength LIMIT 10;")
result.get_as_df()

,d.dimension_id,m.metric_id,r.relationship,r.strength
0,Region,Revenue,Direct,Strong
1,Region,ChurnRiskScore,Inverse,Moderate
2,SalesDate,PurchaseFrequency,Direct,Strong
3,SalesDate,Recency,Inverse,Strong
4,PaymentType,AOV,Direct,Moderate
5,Color,PurchaseFrequency,Direct,Weak
6,Color,StockoutRate,Inverse,Weak
7,ProductCategory,StockoutRate,Direct,Moderate
8,ProductCategory,Revenue,Direct,Strong
9,ProductSubcategory,PurchaseFrequency,Direct,Strong


In [87]:
result = conn.execute("MATCH (u1:User)-[r:Similar_to]->(u2:User) RETURN u1.user_id, u2.user_id, r.similarity_strength LIMIT 10;")
result.get_as_df()


,u1.user_id,u2.user_id,r.similarity_strength
0,1,2,Strong
1,2,3,Strong
2,3,4,Strong
3,5,6,Strong


In [88]:
result = conn.execute("MATCH (u:User)-[r:Associated]->(m:Metrics) RETURN u.user_id, m.metric_id, r.Relationship LIMIT 10;")
result.get_as_df()


,u.user_id,m.metric_id,r.Relationship
0,1,Revenue,user_domain
1,1,AOV,user_theme
2,1,ChurnRiskScore,user_theme
3,1,PurchaseFrequency,user_domain
4,1,CLTV,user_domain
5,2,ReturnRate,user_theme
6,2,StockoutRate,user_domain
7,2,BackOrderRatio,user_domain
8,2,ChurnRiskScore,user_domain
9,2,CLTV,user_domain
